# Permutation of source time courses for testing

In [1]:
import os
from pathlib import Path
import sys
from mne.datasets import fetch_fsaverage, sample
from tabulate import tabulate
import random
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from mne import Epochs, compute_covariance, find_events, make_ad_hoc_cov
from mne.simulation import add_ecg, add_eog, add_noise, simulate_raw, simulate_sparse_stc

fs_dir = fetch_fsaverage(verbose=True)
subject = "fsaverage"
subjects_dir = os.path.dirname(fs_dir)
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
sys.path.append("C:/Users/EISERK01/eeg-projects/Code/eeg_toolkit")
from eeg_toolkit import preprocess, utils
import eeg_toolkit.functional_connectivity as fc
import eeg_toolkit.permutation_testing as pt

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage
0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


### Paths

In [2]:
# Settings
times_tup, time_win_path = preprocess.get_time_window(5)

#preprocessing method
sl_method = 'MNE'

# data paths
data_dir = Path("../../Data")
processed_data_path = data_dir / "Processed Data"
stc_path = data_dir / f"Source Time Courses ({sl_method})"
EO_resting_data_path = stc_path / "Eyes Open"
EC_resting_data_path = stc_path / "Eyes Closed"
zscored_epochs_data_path = stc_path / "zscored_Epochs" / "noise_var" / time_win_path

[-2.5,0.0,2.5]


### Manually enter age/sex matched subjects

In [3]:
# All CP and HC subs
sub_ids_CP = [  
    '018',  
    '020',  
    '021',  
    '027',  
    '031',  
    '032',  
    '033',  
    '035',  
    '036',  
    '037',  
    '038',  
    '040',  
    '043',  
    '044',  
    '048',  
    '049',  
    '050',  
    '057',  
    '059',
]

sub_ids_HC = [
    'C10',  
    'C11',  
    'C12',  
    'C13',  
    'C14',  
    'C15',  
    'C16',  
    'C17',  
    'C18',  
    'C19',  
    'C2.',  
    'C24',  
    'C25',  
    'C26',  
    'C27',  
    'C3.',  
    'C6.',  
    'C7.',  
    'C9.'   
]
  
sub_ids = sub_ids_CP + sub_ids_HC

# CP Subs with widespread pain
sub_ids_WSP = [
    '018',
    '022',
    '024',
    '031',
    '032',
    '034',
    '036',
    '039',
    '040',
    '045',
    '046',
    '052',
]

# CP Subs with localized pain
sub_ids_LP = [  
    '020',  
    '021',  
    '023',  
    '029',  
    '031',  
    '037',  
    '041',  
    '042',  
    '048',  
    '049',  
    '050',  
    '056'  
]  

### Low Back Pain Laterality

In [4]:
left_pain_ids = [
    '016',
    '017',
    '018',
    '019',
    '020',
    '023',
    '024',
    '025',
    '027',
    '028',
    '029',
    '030',
    '032',
    '033',
    '036',
    '038',
    '045',
    '052',
    '054',
    '055',
    '057',
    '058',
]

right_pain_ids = [
    '021',
    '026',
    '031',
    '037',
    '039',
    '043',
    '044',
    '046',
    '047',
    '048',
    '049',
    '050',
    '051',
    '053',
    '056',
]

bilateral_pain_ids = [
    '022',
    '034',
    '035',
    '040',
    '041',
    '042',
    '059',
]

# Randomly assign healthy subjects to left/right/bilateral pain
lst = sub_ids_HC
random.seed(42)
left_pain_ids.extend(random.sample(lst, 8))
lst = [i for i in lst if i not in left_pain_ids]  

# Randomly assign 8 elements to right and remove them from lst  
right_pain_ids.extend(random.sample(lst, 8))
lst = [i for i in lst if i not in right_pain_ids]  
  
# Assign the remaining elements to bilateral  
bilateral_pain_ids.extend(lst)  

In [5]:
for sub_ids, group in zip([sub_ids_CP, sub_ids_WSP, sub_ids_LP, sub_ids_HC],"CP WSP LP HC".split()):
    print(f"\n{group}")
    for sub_pain_ids, laterality in zip([left_pain_ids, right_pain_ids, bilateral_pain_ids],"Left Right Bilateral".split()):
        print(f"{laterality} - {len(set(sub_ids) & set(sub_pain_ids))}")
        print(set(sub_ids) & set(sub_pain_ids))


CP
Left - 8
{'032', '038', '020', '018', '033', '057', '036', '027'}
Right - 8
{'044', '037', '048', '050', '049', '043', '021', '031'}
Bilateral - 3
{'035', '059', '040'}

WSP
Left - 6
{'032', '052', '024', '045', '018', '036'}
Right - 3
{'039', '046', '031'}
Bilateral - 3
{'034', '022', '040'}

LP
Left - 3
{'029', '020', '023'}
Right - 7
{'048', '037', '056', '050', '049', '021', '031'}
Bilateral - 2
{'041', '042'}

HC
Left - 8
{'C10', 'C12', 'C18', 'C13', 'C9.', 'C17', 'C24', 'C11'}
Right - 8
{'C7.', 'C26', 'C27', 'C6.', 'C3.', 'C14', 'C15', 'C25'}
Bilateral - 3
{'C2.', 'C16', 'C19'}


In [6]:
# Exclude HEALTHY subjects with widespread pain
widespread_pain_HC = ['C1.','C23']
sub_ids_HC = [sub for sub in sub_ids_HC if sub not in widespread_pain_HC]

In [7]:
print(f"Chronics (all): {len(sub_ids_CP)}")
print([el for el in sub_ids_CP])
print(f"Chronics with widespread pain: {len(sub_ids_WSP)}")
print([el for el in sub_ids_WSP])
print(f"Chronics with localized pain: {len(sub_ids_LP)}")
print([sub for sub in sub_ids_LP])
print(f"Controls: {len(sub_ids_HC)}")
print([el for el in sub_ids_HC])
print(f"\nTotal (Chronics + Controls): {len(sub_ids)}")

Chronics (all): 19
['018', '020', '021', '027', '031', '032', '033', '035', '036', '037', '038', '040', '043', '044', '048', '049', '050', '057', '059']
Chronics with widespread pain: 12
['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052']
Chronics with localized pain: 12
['020', '021', '023', '029', '031', '037', '041', '042', '048', '049', '050', '056']
Controls: 19
['C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']

Total (Chronics + Controls): 19


In [8]:
sim_sub_ids = ['018'] #['018', 'C10', '052', '056']

In [9]:
for sub_id in sim_sub_ids:
    # Load the original z-scored epochs data, eyes open data, eyes closed data
    label_ts = utils.unpickle_data(zscored_epochs_data_path, f"{sub_id}_epochs.pkl")
    eo_data = utils.unpickle_data(EO_resting_data_path, f"{sub_id}_eyes_open.pkl")

    # For each epoch, make all of the regions identical to the first
    for i in range(0, len(label_ts)):
        label_ts[i][:] = label_ts[i][0]
        
    # Make all the regions in the resting data equal to the first
    eo_data[:] = eo_data[0]
                
    # Save the simulated data
    subset = "Simulated Data"
    #make zcored_epochs save path
    fc_subset_path = stc_path / subset / "zscored_Epochs" / time_win_path
    save_path = fc_subset_path
    os.makedirs(save_path, exist_ok=True)
    utils.pickle_data(save_path, f"{sub_id}_epochs.pkl", label_ts)

    #make eyes open save path
    fc_subset_path = stc_path / subset / "Eyes Open"
    save_path = fc_subset_path
    os.makedirs(save_path, exist_ok=True)
    utils.pickle_data(save_path, f"{sub_id}_eyes_open.pkl", eo_data)

    # Clear display
    utils.clear_display()

Saved 018_epochs.pkl to ../../Data/Source Time Courses (MNE)/Simulated Data/zscored_Epochs/5_sec_time_window.
Saved 018_eyes_open.pkl to ../../Data/Source Time Courses (MNE)/Simulated Data/Eyes Open.


### Permute source space time signal, repeat 100

In [10]:
repetitions = 1 #takes forever, switch to a smaller number for testing

# data paths
data_dir = Path("../../Data")
processed_data_path = data_dir / "Processed Data"
stc_path = data_dir / f"Source Time Courses ({sl_method})/Simulated Data"
EO_resting_data_path = stc_path / "Eyes Open"
EC_resting_data_path = stc_path / "Eyes Closed"
zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

In [11]:
# Settings
sfreq = 400
tmin=0.0
tmax=1.25

In [12]:
for j in range(0, repetitions):
# Loop over each group
    for this_sub_ids in [
        sim_sub_ids
        ]:
        for sub_id in this_sub_ids:
            count = 1

            # Load the original z-scored epochs data, eyes open data, eyes closed data
            print(zscored_epochs_data_path, f"{sub_id}_epochs.pkl")
            label_ts = utils.unpickle_data(zscored_epochs_data_path, f"{sub_id}_epochs.pkl")

            # Permute the signals one epoch at a time
            for i,el in enumerate(label_ts):
                # print(f"Epoch {i}:",el.shape)
                print(el)
                # Create the surrogate data for zscored_epochs
                surrogate_data = pt.create_surrogate_data(el,sfreq,tmin,tmax)
                print(surrogate_data)
                
                # # Replace the original data with the surrogate data

            exit()
            
            # eo_data = utils.unpickle_data(EO_resting_data_path, f"{sub_id}_eyes_open.pkl")
            # print(EO_resting_data_path, f"{sub_id}_eyes_open.pkl")
            
            # # Create the surrogate data for eyes open, replace original data with surrogate
            # surrogate_data = pt.create_surrogate_data(eo_data)
            # eo_data = surrogate_data
            
            # # Save the surrogate data
            # subset = "FFT Permuted Data " + str(j+1)
            # #make zcored_epochs save path
            # fc_subset_path = stc_path / subset / "zscored_Epochs" / time_win_path
            # save_path = fc_subset_path
            # os.makedirs(save_path, exist_ok=True)
            # utils.pickle_data(save_path, f"{sub_id}_epochs.pkl", label_ts)

            # #make eyes open save path
            # fc_subset_path = stc_path / subset / "Eyes Open"
            # save_path = fc_subset_path
            # os.makedirs(save_path, exist_ok=True)
            # utils.pickle_data(save_path, f"{sub_id}_eyes_open.pkl", eo_data)

            # # Clear display
            # utils.clear_display()
            # # break

../../Data/Source Time Courses (MNE)/Simulated Data/zscored_Epochs/5_sec_time_window 018_epochs.pkl
[[7.39454044e-07 5.89032071e-07 3.92975707e-07 ... 6.68545765e-07
  4.96305143e-07 3.49553572e-07]
 [7.39454044e-07 5.89032071e-07 3.92975707e-07 ... 6.68545765e-07
  4.96305143e-07 3.49553572e-07]
 [7.39454044e-07 5.89032071e-07 3.92975707e-07 ... 6.68545765e-07
  4.96305143e-07 3.49553572e-07]
 ...
 [7.39454044e-07 5.89032071e-07 3.92975707e-07 ... 6.68545765e-07
  4.96305143e-07 3.49553572e-07]
 [7.39454044e-07 5.89032071e-07 3.92975707e-07 ... 6.68545765e-07
  4.96305143e-07 3.49553572e-07]
 [7.39454044e-07 5.89032071e-07 3.92975707e-07 ... 6.68545765e-07
  4.96305143e-07 3.49553572e-07]]
len(data_to_permute) = 12
[[ 5.02788600e-07  5.50794351e-07  6.59507172e-07 ...  1.03191091e-06
   1.07529612e-06  8.08679061e-07]
 [ 1.21831520e-07  1.73569390e-07  2.27573635e-07 ... -2.27387084e-07
   9.92896112e-10  1.71847038e-07]
 [ 1.24450002e-06  1.23799731e-06  1.14660876e-06 ...  8.1199946

: 

### Settings for conditions/bands/methods

In [ ]:
# Settings

# Include intermediate stimulus?
include_LS = False

# Data properties
sfreq = 400  # Hz

# Choose which connectivity estimates to run. "amplitude", "phase", or "both"
# method_choice = "phase"
method_choice = "both"

# Load previously saved group data
# load_group_data_flag = True
load_group_data_flag = False

# Save group data?
save_group_data_flag = False
# save_group_data_flag = True
   
# Evoked only or include resting too?
# include_resting = False
include_resting = True

# Orthogonalize AEC?
orthogonalize_AEC = False


### Define ROIs, frequency bands, conditions, and methods for FC

In [ ]:
####################################################################
# REGIONS OF INTEREST
roi_names = [  # Left
    "rostralanteriorcingulate-lh",  # Left Rostral ACC
    "caudalanteriorcingulate-lh",  # Left Caudal ACC
    "postcentral-lh",  # Left S1,
    "insula-lh",
    "superiorfrontal-lh",  # Left Insula, Left DL-PFC,
    "medialorbitofrontal-lh",  # Left Medial-OFC
    # Right
    "rostralanteriorcingulate-rh",  # Right Rostral ACC
    "caudalanteriorcingulate-rh",  # Right Caudal ACC
    "postcentral-rh",  # , Right S1
    "insula-rh",
    "superiorfrontal-rh",  # Right Insula, Right DL-PFC
    "medialorbitofrontal-rh",
]  # Right Medial-OFC

# Write out ROI names as acronyms
roi_acronyms = ["rACC-lh", "dACC-lh", "S1-lh", "insula-lh", "dlPFC-lh", "mOFC-lh",
                "rACC-rh", "dACC-rh", "S1-rh", "insula-rh", "dlPFC-rh", "mOFC-rh"]

####################################################################
# BANDS OF INTEREST
Freq_Bands = {
    # 'delta': [0, 4],
    "theta": [4.0, 8.0],
    "alpha": [8.0, 13.0],
    "beta": [13.0, 30.0],
    "low-gamma": [30.0, 58.5],
    # "notch": [58.5, 61.5],
    "high-gamma": [61.5, 100.0],
}

band_names = [band for band in Freq_Bands]

####################################################################
# CONNECTIVITY METHODS
if method_choice == "phase":
    con_methods = ["wpli2_debiased",
                ]
elif method_choice == "amplitude":
    con_methods = [
        # "aec_pairwise",
        "aec_symmetric",
        ]
elif method_choice == "both":
    con_methods = [
    "wpli2_debiased",
    # "aec_pairwise",
    "aec_symmetric", # keep only symmetric for now
    ]
    
####################################################################
# CONDITIONS
conditions = (
    [
        "Hand 32 mN",
        "Hand 128 mN",
        "Hand 256 mN",
        "Back 32 mN",
        "Back 128 mN",
        "Back 256 mN",
        "Eyes Open",
        "Eyes Closed",
    ]
    if include_LS
    else [
        # "Hand 32 mN",
        # 'Hand LS',
        "Hand 256 mN",
        # "Back 32 mN",
        # 'Back LS',
        # "Back 256 mN",
        "Eyes Open",
        # "Eyes Closed",
    ]
)

#Choose to exclude resting state data
conditions = conditions if include_resting else [
    c for c in conditions if "Eyes" not in c
]

####################################################################
# CONDITION DICTIONARY
condition_dict = {
        "Hand 32 mN": 0,
        'Hand LS': 1,
        "Hand 256 mN": 2,
        "Back 32 mN": 3,
        'Back LS': 4,
        "Back 256 mN": 5,
        "Eyes Open": 6,
        "Eyes Closed": 7,
}

### Compute functional connectivity for each subject in each group (surrogate data)

In [ ]:
all_sub_con_CP = []
sub_ids_CP = sim_sub_ids

# loop through all surrogate data
for i in range(0, repetitions):
    processed_data_path = data_dir / "Processed Data"
    stc_parent_path = data_dir / f"Source Time Courses ({sl_method})/Simulated Data"
    stc_path = stc_parent_path / f"FFT Permuted Data {i+1}"
    print(stc_path)
    EO_resting_data_path = stc_path / "Eyes Open"
    EC_resting_data_path = stc_path / "Eyes Closed"
    zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

    # Chronics (all) group
    if not load_group_data_flag:
        sub_con_CP = {}
        for sub_id in sub_ids_CP:
            #print(sub_id)
            sub_avg_cons = fc.compute_sub_avg_con(
                sub_id,
                "Chronic Pain",
                processed_data_path,
                zscored_epochs_data_path,
                EO_resting_data_path,
                EC_resting_data_path,
                con_methods,
                conditions,
                condition_dict,
                roi_names,
                roi_acronyms,
                Freq_Bands,
                sfreq,
                orthogonalize_AEC=orthogonalize_AEC,
                left_pain_ids=left_pain_ids,
                right_pain_ids=right_pain_ids,
                bilateral_pain_ids=bilateral_pain_ids,
                include_LS=include_LS,
            )
            
            sub_con_CP[sub_id] = sub_avg_cons
        all_sub_con_CP.append(sub_con_CP)
    utils.clear_display()

: 

In [ ]:
all_sub_con_HC = []
sub_ids_HC = sim_sub_ids

# loop through all surrogate data
for i in range(0, repetitions):
    processed_data_path = data_dir / "Processed Data"
    stc_parent_path = data_dir / f"Source Time Courses ({sl_method})/Simulated Data"
    stc_path = stc_parent_path / f"FFT Permuted Data {i+1}"
    print(stc_path)
    EO_resting_data_path = stc_path / "Eyes Open"
    EC_resting_data_path = stc_path / "Eyes Closed"
    zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

    # Healthy Control group
    if not load_group_data_flag:
        sub_con_HC = {}
        for sub_id in sub_ids_HC:
            #print(sub_id)
            sub_avg_cons = fc.compute_sub_avg_con(
                sub_id,
                "Healthy Controls",
                processed_data_path,
                zscored_epochs_data_path,
                EO_resting_data_path,
                EC_resting_data_path,
                con_methods,
                conditions,
                condition_dict,
                roi_names,
                roi_acronyms,
                Freq_Bands,
                sfreq,
                orthogonalize_AEC=orthogonalize_AEC,
                left_pain_ids=left_pain_ids,
                right_pain_ids=right_pain_ids,
                bilateral_pain_ids=bilateral_pain_ids,
                include_LS=include_LS,
            )
            sub_con_HC[sub_id] = sub_avg_cons
        all_sub_con_HC.append(sub_con_HC)
    utils.clear_display()

In [ ]:
all_group_con_HC = []
all_group_con_CP = []
#all_group_con_WSP = []
#all_group_con_LP = []

for rep in range(0, repetitions):
    print(rep)
    all_group_con_HC.append(fc.compute_group_con(all_sub_con_HC[rep], conditions, con_methods, band_names))
    all_group_con_CP.append(fc.compute_group_con(all_sub_con_CP[rep], conditions, con_methods, band_names))
    #all_group_con_WSP.append(fc.compute_group_con(all_sub_con_WSP[rep], conditions, con_methods, band_names))
    #all_group_con_LP.append(fc.compute_group_con(all_sub_con_LP[rep], conditions, con_methods, band_names))

In [ ]:
# Highlight p-values with red box automatically
highlight_pvals=False

# Make any non-significant squares white
show_only_significant=True

# Set title automatically
set_title=True

# Show values in the matrix quadrants
show_fc_vals=True

# *KEEP FALSE* Round negative values to zero 
# (Vinck. et al 2011 has negative values for WPLI 2 Debiased. NO ROUNDING)
round_neg_vals=False

# Plot 3D brain visualization (for AEC only)
plot_brain=False

# Save tables to txt files
save_txt=False
# save_txt=True

### Compute functional connectivity for each subject in each group (original data)

In [ ]:
processed_data_path = data_dir / "Processed Data"
stc_path = data_dir / f"Source Time Courses ({sl_method})/Simulated Data"
EO_resting_data_path = stc_path / "Eyes Open"
EC_resting_data_path = stc_path / "Eyes Closed"
zscored_epochs_data_path = stc_path / "zscored_Epochs" / time_win_path

In [ ]:
# All Chronic Pain
sub_ids_CP = sim_sub_ids

if not load_group_data_flag:
    sub_con_CP = {}
    for sub_id in sub_ids_CP:
        print(sub_id)
        sub_avg_cons = fc.compute_sub_avg_con(
            sub_id,
            "Chronic Pain",
            processed_data_path,
            zscored_epochs_data_path,
            EO_resting_data_path,
            EC_resting_data_path,
            con_methods,
            conditions,
            condition_dict,
            roi_names,
            roi_acronyms,
            Freq_Bands,
            sfreq,
            orthogonalize_AEC=orthogonalize_AEC,
            left_pain_ids=left_pain_ids,
            right_pain_ids=right_pain_ids,
            bilateral_pain_ids=bilateral_pain_ids,
            include_LS=include_LS,
        )
        sub_con_CP[sub_id] = sub_avg_cons            
        
    utils.clear_display()

In [ ]:
# All Healthy Controls
sub_ids_HC = sim_sub_ids

if not load_group_data_flag:
    sub_con_HC = {}
    for sub_id in sub_ids_HC:
        print(sub_id)
        sub_avg_cons = fc.compute_sub_avg_con(
            sub_id,
            "Healthy Controls",
            processed_data_path,
            zscored_epochs_data_path,
            EO_resting_data_path,
            EC_resting_data_path,
            con_methods,
            conditions,
            condition_dict,
            roi_names,
            roi_acronyms,
            Freq_Bands,
            sfreq,
            orthogonalize_AEC=orthogonalize_AEC,
            left_pain_ids=left_pain_ids,
            right_pain_ids=right_pain_ids,
            bilateral_pain_ids=bilateral_pain_ids,
            include_LS=include_LS,
        )
        sub_con_HC[sub_id] = sub_avg_cons            
        
    utils.clear_display()

In [ ]:
group_con_HC = fc.compute_group_con(sub_con_HC, conditions, con_methods, band_names)
group_con_CP = fc.compute_group_con(sub_con_CP, conditions, con_methods, band_names)
#group_con_WSP = fc.compute_group_con(sub_con_WSP, conditions, con_methods, band_names)
#group_con_LP = fc.compute_group_con(sub_con_LP, conditions, con_methods, band_names)

### Testing wpli

In [ ]:
label_ts = utils.unpickle_data(zscored_epochs_data_path, f"{sub_id}_epochs.pkl")
#make all of the epochs the same for each condition
for i in range(0, len(label_ts)):
    for j in range(0, len(label_ts[i])):
        label_ts[i][j] = label_ts[i][0]

In [ ]:
print(len(label_ts), len(label_ts[0]), len(label_ts[0][0]))

In [ ]:
for j in range(0, len(label_ts)):
    string = ''
    for i in range(0, len(label_ts[0])):
        string = string + str(label_ts[j][i][0]) + ' '
    print(string)

In [ ]:
ex_data = np.array([[ex_signal]*12]*1)
print(ex_data.shape)

fmins = [Freq_Bands[f][0] for f in Freq_Bands]
fmaxs = [Freq_Bands[f][1] for f in Freq_Bands]
for fmin, fmax, band_name in zip(fmins, fmaxs, Freq_Bands):
    con = con_epochs = mne_conn.spectral_connectivity_epochs(
        ex_data,
        method='wpli2_debiased',
        mode="multitaper",
        sfreq=sfreq,
        fmin=fmin,
        fmax=fmax,
        tmin=0.0,
        tmax=1.25,
        faverage=True,
        mt_adaptive=True,
        n_jobs=1,
        )
    data = con.get_data()
    data = data.reshape(len(roi_names), len(roi_names))
    print(data)

In [ ]:
ex_data = np.array([[ex_signal[:401]]*12]*1)
print(ex_data.shape)

fmins = [Freq_Bands[f][0] for f in Freq_Bands]
fmaxs = [Freq_Bands[f][1] for f in Freq_Bands]

for fmin, fmax, band_name in zip(fmins, fmaxs, Freq_Bands):
    freqs = np.linspace(fmin, fmax, int((fmax - fmin) * 4 + 1))
    
    con = con_epochs = mne_conn.spectral_connectivity_time(
        ex_data,
        freqs=freqs,
        method='wpli2_debiased',
        mode="multitaper",
        sfreq=sfreq,
        fmin=fmin,
        fmax=fmax,
        faverage=True,
        n_jobs=1,
        )
    data = con.get_data()
    data = data.reshape(len(roi_names), len(roi_names))
    print(data)

In [ ]:
label_ts = utils.unpickle_data(zscored_epochs_data_path, f"{sub_id}_epochs.pkl")
print(label_ts)
#make all of resting data the same for each region
#for i,el in enumerate(label_ts):
#    label_ts[i] = label_ts[0]

In [ ]:
for j in range(0, len(label_ts)):
    string = ''
    for i in range(0, len(label_ts[0])):
        string = string + str(label_ts[j][i][0]) + ' '
    print(string)

In [ ]:
con = con_epochs = mne_conn.spectral_connectivity_epochs(
    label_ts,
    method='wpli2_debiased',
    mode="multitaper",
    sfreq=sfreq,
    fmin=fmin,
    fmax=fmax,
    tmin=tmin,
    tmax=tmax,
    faverage=True,
    mt_adaptive=True,
    n_jobs=1,
    )
data = con.get_data()
data = data.reshape(len(roi_names), len(roi_names))
print(data)

### CP vs. HC Mann-Whitney

In [ ]:
# change S1-lh and S1-rh in roi_acronyms to S1-i and S1-c
roi_acronyms_resting = roi_acronyms.copy()
roi_acronyms_resting[roi_acronyms.index("S1-lh")] = "S1-i"
roi_acronyms_resting[roi_acronyms.index("S1-rh")] = "S1-c"

In [ ]:
# Chronic Pain vs Healthy Control
these_group_names = ["Chronic Pain", "Healthy Control", "Mann_Whitney_U_test"]
these_group_cons = [group_con_CP, group_con_HC]
these_sub_cons = [sub_con_CP, sub_con_HC]
these_sub_ids = [sub_ids_CP, sub_ids_HC]

sig_pairs_CP_HC = {}

for condition in conditions:

    sig_pairs_CP_HC[condition] = {}

    for method in con_methods:

        sig_pairs_CP_HC[condition][method] = {}

        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="wpli2_debiased":
            continue
        
        for band in band_names:
            # Select groups for mann-whitney
            print(f"{condition} - {these_group_names[0]} vs. {these_group_names[1]} - {method} - {band}")
        
            # For each of the top 3 connections, add the connection, its frequency, and its mean strength to a list  
            top_3_info=None
            group_con=None
            for group_con,group_name,sub_con in zip(these_group_cons, these_group_names, these_sub_cons):
                table_data = []  
                top_3_info = group_con[condition][method][band]["top 3"] 
                print(f"Top 3 Connections in {group_name} group")
                for i in range(3):  
                    connection = top_3_info["connections"][i]  
                    frequency = top_3_info["frequency"][i]  
                    mean_strength = top_3_info["mean strength"][i]
                    if 'Eyes' in condition:
                        table_data.append([f"{roi_acronyms_resting[connection[0]]} <-> {roi_acronyms_resting[connection[1]]}", f"{frequency}/{len(sub_con)}", mean_strength]) 
                    else:
                        table_data.append([f"{roi_acronyms[connection[0]]} <-> {roi_acronyms[connection[1]]}", f"{frequency}/{len(sub_con)}", mean_strength]) 
                        
                # Define table headers  
                headers = ['Connection', 'Frequency', 'Mean Strength']  
                
                # Print the table  
                print(tabulate(table_data, headers, tablefmt='pretty'),'\n')  

            group1_stack = these_group_cons[0][condition][method][band]["data"]
            print(group1_stack.shape)
            group2_stack = these_group_cons[1][condition][method][band]["data"]

            nepochs_1 = these_group_cons[0][condition]['num_epochs']
            nepochs_2 = these_group_cons[1][condition]['num_epochs']
            
            # Compute the Mann-Whitney U test
            p_values, means_1, sem_1, means_2, sem_2 = fc.mann_whitney_test(
                group1_stack,
                group2_stack,
                roi_acronyms=roi_acronyms if 'Eyes' not in condition else roi_acronyms_resting,
                sub_ids1=these_sub_ids[0],
                sub_ids2=these_sub_ids[1],
                condition=condition,
                bilateral_pain_ids=bilateral_pain_ids,
                round_neg_vals=round_neg_vals,
                method=method)
            
            #print(p_values)

            # Need to custom set this because the colors blow out
            if 'Eyes' in condition and band in ["theta","alpha"]:
                vmin = 0.0
                vmax = 1.0
            elif 'Eyes' in condition and band not in ["theta","alpha"]:
                vmin = 0.0
                vmax = 0.5
            else:
                vmin = None
                vmax = None
            
            # Plotting
            # Gather the information for plotting
            plot_kwargs= dict(
                method=method,
                band=band, 
                roi_names=roi_names,
                roi_acronyms=roi_acronyms if 'Eyes' not in condition else roi_acronyms_resting,
                condition=condition,
                vmin=vmin,
                vmax=vmax,
                group_names=these_group_names,
                nepochs=[nepochs_1, nepochs_2],
                titles=[these_group_names[0], 
                        these_group_names[1],
                        "WSP vs. LP",],
                save_names=["Mann_Whitney_U_test",
                            these_group_names[0], 
                            these_group_names[1],
                            ],
                save_path='',
                )
 
            print(f"\nNum epochs: {nepochs_1} vs. {nepochs_2}")               
            sig_pairs_CP_HC[condition][method][band] = fc.plot_connectivity_and_stats(means_1=means_1,
                                        means_2=means_2,
                                        sem_1=sem_1,
                                        sem_2=sem_2,
                                        p_values=p_values,
                                        **plot_kwargs,
                                        save_fig=True,
                                        highlight_pvals=highlight_pvals,
                                        min_fc_val=None,
                                        set_title=set_title,
                                        show_fc_vals=show_fc_vals,
                                        round_neg_vals=round_neg_vals,
            )       

In [ ]:
sig_pairs_WSP_LP = {}
sig_pairs_CP_HC = {}

for condition in conditions:

    sig_pairs_WSP_LP[condition] = {}
    sig_pairs_CP_HC[condition] = {}

    for method in con_methods:

        sig_pairs_WSP_LP[condition][method] = {}
        sig_pairs_CP_HC[condition][method] = {}

        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="aec":#"wpli2_debiased":
            continue

        for band in band_names:
            sig_pairs_WSP_LP[condition][method][band] = []
            sig_pairs_CP_HC[condition][method][band] = []
            for roi1 in range(0, len(roi_acronyms)):
                for roi2 in range(0, roi1):
                    if 'Eyes' in condition:
                        sig_pairs_WSP_LP[condition][method][band].append([roi_acronyms_resting[roi1], roi_acronyms_resting[roi2], 0.0, 0.0, 0.0])
                        sig_pairs_CP_HC[condition][method][band].append([roi_acronyms_resting[roi1], roi_acronyms_resting[roi2], 0.0, 0.0, 0.0])
                    else:
                        sig_pairs_WSP_LP[condition][method][band].append([roi_acronyms[roi1], roi_acronyms[roi2], 0.0, 0.0, 0.0])
                        sig_pairs_CP_HC[condition][method][band].append([roi_acronyms[roi1], roi_acronyms[roi2], 0.0, 0.0, 0.0])
            

In [ ]:
#testtest
#not a necessary block of code, just testing
for condition in conditions:
    for method in con_methods:
        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="aec":#wpli2_debiased":
            continue
        for band in band_names:
            print(f"{condition} - {method} - {band}")
            for i in range(0, len(sig_pairs_CP_HC[condition][method][band])):
                print(sig_pairs_CP_HC[condition][method][band][i])
            print()

In [ ]:
#testtest
#not a necessary block of code, just testing
for condition in conditions:
    for method in con_methods:
        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="aec"#"wpli2_debiased":
            continue
        for band in band_names:
            print(f"{condition} - {method} - {band}")
            for i in range(0, len(sig_pairs_WSP_LP[condition][method][band])):
                print(sig_pairs_WSP_LP[condition][method][band][i])
            print()

### Compare original and surrogate connectivity values -- CP vs. HC

In [ ]:
new_pvals_CP_HC = {}

for condition in conditions:

    new_pvals_CP_HC[condition] = {}

    for method in con_methods:
        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="wpli2_debiased":
            continue

        new_pvals_CP_HC[condition][method] = {}

        for band in band_names:

            new_pvals_CP_HC[condition][method][band] = {}

            for i in range(0, len(sig_pairs_CP_HC[condition][method][band])):
                roi1 = sig_pairs_CP_HC[condition][method][band][i][0]
                roi2 = sig_pairs_CP_HC[condition][method][band][i][1]
                pval = sig_pairs_CP_HC[condition][method][band][i][2]
                mean1 = sig_pairs_CP_HC[condition][method][band][i][3]
                mean2 = sig_pairs_CP_HC[condition][method][band][i][4]

                roi_index1 = 0
                roi_index2 = 0

                if 'Eyes' in condition:
                    roi_index1 = roi_acronyms_resting.index(roi1)
                    roi_index2 = roi_acronyms_resting.index(roi2)
                else:
                    roi_index1 = roi_acronyms.index(roi1)
                    roi_index2 = roi_acronyms.index(roi2)
                
                #print(condition, method, band, roi_index1, roi_index2, pval, mean1, mean2)
                new_pvals_CP_HC[condition][method][band][roi_index1] = {}
                new_pvals_CP_HC[condition][method][band][roi_index1][roi_index2] = {}

                # Gather connectivity info for whichever mean is greater
                if mean1 > mean2:
                    sub_ids = sub_ids_CP
                    sub_con = sub_con_CP
                    all_sub_con = all_sub_con_CP
                else:
                    sub_ids = sub_con_HC
                    sub_con = sub_con_HC
                    all_sub_con = all_sub_con_HC

                for sub_id in sub_ids:
                    # Find this subject's original data connectivity for this ROI pair
                    orig_connectivity = sub_con[sub_id][condition][method][band]['data'][roi_index1][roi_index2]
                    # Gather connectivities from this subject's surrogate data for this ROI pair
                    all_connectivities = []
                    # Keep count of how many surrogate data connectivity values are greater than the original connectivity
                    greater_than = 0
                    for i in range(0, repetitions):
                        temp_conn = all_sub_con[i][sub_id][condition][method][band]['data'][roi_index1][roi_index2]
                        if temp_conn > orig_connectivity:
                            greater_than += 1
                        all_connectivities.append(temp_conn)
                    # Add the fraction of surr. connectivities > orig. connectivity, the orig. value, and array of the surrogate values to dictionary
                    new_pvals_CP_HC[condition][method][band][roi_index1][roi_index2][sub_id] = [greater_than, orig_connectivity, all_connectivities]

    

In [ ]:
with PdfPages('CP_HC_output_figures_bin-num20.pdf') as pdf:
    for condition in conditions:  
        for method in con_methods:
            # Ignore some specific condition/method combinations
            if condition=="Hand 256 mN" and "aec" in method:
                continue
            elif condition=="Eyes Open" and method=="wpli2_debiased":
                continue
            for band in band_names:
                temp = new_pvals_CP_HC[condition][method][band]
                for roi1 in temp.keys():
                    temp2 = temp[roi1]
                    for roi2 in temp2.keys():
                        temp3 = temp2[roi2]

                        plt.figure(figsize=(16,12))
                        plt.suptitle('CP vs. HC - ' + condition + ' - ' + method + ' - ' + band + ' - ' + roi_acronyms_resting[roi1] + '<->' + roi_acronyms_resting[roi2], y=0.99)
                        count = 1
                        for sub_id in temp3.keys():
                            greater_than = temp3[sub_id][0]
                            orig_conn = temp3[sub_id][1]
                            connectivities = temp3[sub_id][2]
                            connectivities.sort()

                            plt.subplot(4,5,count)
                            plt.title(sub_id)
                            plt.hist(connectivities, 20, (connectivities[0], connectivities[len(connectivities)-1]))
                            plt.axvline(x=orig_conn, color='red', label=round(orig_conn,3))
                            plt.text(connectivities[0]+0.01, 0.5, str(greater_than)+' surr. > orig.')
                            plt.legend()
                            
                            count += 1
                        plt.tight_layout()
                        pdf.savefig()
                        plt.show()
                        plt.close()
               

### Compare original and surrogate connectivity values -- WSP vs. LP

In [ ]:
new_pvals_WSP_LC = {}

for condition in conditions:

    new_pvals_WSP_LC[condition] = {}
    
    for method in con_methods:
        # Ignore some specific condition/method combinations
        if condition=="Hand 256 mN" and "aec" in method:
            continue
        elif condition=="Eyes Open" and method=="wpli2_debiased":
            continue

        new_pvals_WSP_LC[condition][method] = {}

        for band in band_names:

            new_pvals_WSP_LC[condition][method][band] = {}

            for i in range(0, len(sig_pairs_WSP_LP[condition][method][band])):
                roi1 = sig_pairs_WSP_LP[condition][method][band][i][0]
                roi2 = sig_pairs_WSP_LP[condition][method][band][i][1]
                pval = sig_pairs_WSP_LP[condition][method][band][i][2]
                mean1 = sig_pairs_WSP_LP[condition][method][band][i][3]
                mean2 = sig_pairs_WSP_LP[condition][method][band][i][4]

                roi_index1 = 0
                roi_index2 = 0

                if 'Eyes' in condition:
                    roi_index1 = roi_acronyms_resting.index(roi1)
                    roi_index2 = roi_acronyms_resting.index(roi2)
                else:
                    roi_index1 = roi_acronyms.index(roi1)
                    roi_index2 = roi_acronyms.index(roi2)
                
                #print(condition, method, band, roi_index1, roi_index2, pval, mean1, mean2)
                new_pvals_WSP_LC[condition][method][band][roi_index1] = {}
                new_pvals_WSP_LC[condition][method][band][roi_index1][roi_index2] = {}

                # Gather connectivity info for whichever mean is greater
                if mean1 > mean2:
                    sub_ids = sub_ids_WSP
                    sub_con = sub_con_WSP
                    all_sub_con = all_sub_con_WSP
                else:
                    sub_ids = sub_ids_LP
                    sub_con = sub_con_LP
                    all_sub_con = all_sub_con_LP
                for sub_id in sub_ids:
                    # Find this subject's original data connectivity for this ROI pair
                    orig_connectivity = sub_con[sub_id][condition][method][band]['data'][roi_index1][roi_index2]
                    # Gather connectivities from this subject's surrogate data for this ROI pair
                    all_connectivities = []
                    # Keep count of how many surrogate data connectivity values are greater than the original connectivity
                    greater_than = 0
                    for i in range(0, repetitions):
                        temp_conn = all_sub_con[i][sub_id][condition][method][band]['data'][roi_index1][roi_index2]
                        if temp_conn > orig_connectivity:
                            greater_than += 1
                        all_connectivities.append(temp_conn)
                        # Add the fraction of surr. connectivities > orig. connectivity, the orig. value, and array of the surrogate values to dictionary
                        new_pvals_WSP_LC[condition][method][band][roi_index1][roi_index2][sub_id] = [greater_than, orig_connectivity, all_connectivities]

In [ ]:
with PdfPages('WSP_LP_output_figures_bin-num5.pdf') as pdf:
    for condition in conditions:
        for method in con_methods:
            # Ignore some specific condition/method combinations
            if condition=="Hand 256 mN" and "aec" in method:
                continue
            elif condition=="Eyes Open" and method=="wpli2_debiased":
                continue
            for band in band_names:
                temp = new_pvals_WSP_LC[condition][method][band]
                for roi1 in temp.keys():
                    temp2 = temp[roi1]
                    for roi2 in temp2.keys():
                        temp3 = temp2[roi2]

                        plt.figure(figsize=(16,12))
                        plt.suptitle('WSP vs. LP - ' + condition + ' - ' + method + ' - ' + band + ' - ' + roi_acronyms_resting[roi1] + '<->' + roi_acronyms_resting[roi2], y=0.99)
                        count = 1
                        for sub_id in temp3.keys():
                            greater_than = temp3[sub_id][0]
                            orig_conn = temp3[sub_id][1]
                            connectivities = temp3[sub_id][2]
                            connectivities.sort()

                            plt.subplot(3,4,count)
                            plt.title(sub_id)
                            plt.hist(connectivities, 10, (connectivities[0], connectivities[len(connectivities)-1]))
                            plt.axvline(x=orig_conn, color='red', label=round(orig_conn,3))
                            plt.text(connectivities[0]+0.01, 1, str(greater_than)+' trials > orig. conn.')
                            plt.legend()
                            
                            count += 1
                        plt.tight_layout()
                        pdf.savefig()
                        plt.show()
                        plt.close()
